In [57]:
import pandas as pd

In [58]:
file_path = 'data/Shipment_Details.csv'
df = pd.read_csv(file_path)
df

,SH_ID,C_ID,SH_CONTENT,SH_DOMAIN,SER_TYPE,SH_WEIGHT,SH_CHARGES,SR_ADDR,DS_ADDR
0,690,230,Healthcare,Domestic,Regular,553,1210,1800 Block of 26TH ST,1200 Block of JACKSON ST
1,933,3189,Healthcare,International,Express,810,1114,2600 Block of ALEMANY BL,700 Block of HAMPSHIRE ST
2,261,2216,Luggage,Domestic,Express,994,1020,BARTLETT ST / 23RD ST,500 Block of HAIGHT ST
3,445,1904,Home Furnishing,Domestic,Express,598,1351,1300 Block of 7TH AV,300 Block of 9TH ST
4,722,7342,Electronics,International,Express,412,566,0 Block of EUREKA ST,1800 Block of VANNESS AV
...,...,...,...,...,...,...,...,...,...
195,955,2104,Home Furnishing,International,Express,957,1007,BARTLETT ST / 21ST ST,FELL ST / POLK ST
196,538,1702,Industrial Equipments,International,Express,484,863,700 Block of MARKET ST,700 Block of SWEENY ST
197,169,8933,Electronics,International,Express,576,1077,0 Block of CAMERON WY,200 Block of DORE ST
198,924,3624,Construction,International,Regular,606,1021,FILLMORE ST / OFARRELL ST,HAWTHORNE ST / HARRISON ST


In [59]:
id_field = 'SH_ID'

# Check how many unique IDs there are
unique_ids = df[id_field].nunique()
unique_ids

200

In [60]:
df.shape[0]

200

In [61]:
df['SH_ID'].iloc[1]

933

## NoSQL synthetic data
 

In [62]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()

True

In [63]:
client = OpenAI(
    api_key = os.getenv("OPENAI_API_KEY")
)

### Random variables

In [69]:
import random
from datetime import datetime, timedelta

# Define the states and status_dict objects
states = {
    1: "Alabama", 2: "Alaska", 3: "Arizona", 4: "Arkansas", 5: "California", 6: "Colorado",
    7: "Connecticut", 8: "Delaware", 9: "Florida", 10: "Georgia", 11: "Hawaii", 12: "Idaho",
    13: "Illinois", 14: "Indiana", 15: "Iowa", 16: "Kansas", 17: "Kentucky", 18: "Louisiana",
    19: "Maine", 20: "Maryland", 21: "Massachusetts", 22: "Michigan", 23: "Minnesota", 
    24: "Mississippi", 25: "Missouri", 26: "Montana", 27: "Nebraska", 28: "Nevada", 
    29: "New Hampshire", 30: "New Jersey", 31: "New Mexico", 32: "New York", 33: "North Carolina",
    34: "North Dakota", 35: "Ohio", 36: "Oklahoma", 37: "Oregon", 38: "Pennsylvania", 
    39: "Rhode Island", 40: "South Carolina", 41: "South Dakota", 42: "Tennessee", 
    43: "Texas", 44: "Utah", 45: "Vermont", 46: "Virginia", 47: "Washington", 48: "West Virginia",
    49: "Wisconsin", 50: "Wyoming"
}

status_dict = {
    1: ["Shipped"],
    2: ["Shipped", "In Transit"],
    3: ["Shipped", "In Transit", "Delivered"]
}

def generate_random_timestamps(num_timestamps, max_days_range=4):
    """
    Generates a list of timestamps where the first event is the oldest and the last event is the most recent.
    The total range between the first and last timestamps will be randomly up to `max_days_range` days.
    The day, month, and year of the timestamps will also be randomized between 2021 and 2023.
    """
    # Define the random total time span (up to max_days_range)
    total_days_range = random.uniform(1, max_days_range)  # Random range from 1 to 4 days

    # Randomly choose a year between 2021 and 2023
    random_year = random.randint(2021, 2023)

    # Randomly choose a month and day within that year
    random_month = random.randint(1, 12)
    random_day = random.randint(1, 28)  # To avoid issues with months having fewer than 31 days

    # Create a random starting timestamp (most recent) based on year, month, and day
    latest_time = datetime(random_year, random_month, random_day)

    # The first timestamp (Shipped) will be `total_days_range` days before the latest_time
    earliest_time = latest_time - timedelta(days=total_days_range)

    # Generate random time points between earliest_time and latest_time
    timestamps = [earliest_time]  # First is the earliest (e.g., "Shipped")

    # Create intermediate timestamps, evenly spread between earliest and latest
    for _ in range(1, num_timestamps - 1):
        random_time = earliest_time + timedelta(
            days=random.uniform(0, total_days_range)
        )
        timestamps.append(random_time)

    timestamps.append(latest_time)  # Last is the most recent (e.g., "Delivered")

    # Sort the timestamps in ascending order to maintain the correct event order
    return sorted(timestamps)

def get_random_state_status_and_timestamps():
    # Get a random state from the states dictionary
    random_state_number = random.randint(1, 50)
    random_state = states[random_state_number]
    
    # Get a random status sequence from the status_dict
    random_status_number = random.randint(1, 3)
    status_sequence = status_dict[random_status_number]
    
    # Generate corresponding timestamps for the status sequence
    timestamps = generate_random_timestamps(len(status_sequence))
    
    # Create a list of statuses with corresponding timestamps in the desired format
    statuses = [
        {
            "status": status,
            "timestamp": timestamp.isoformat() + "Z"  # Format timestamp in ISO format with 'Z' at the end
        }
        for status, timestamp in zip(status_sequence, timestamps)
    ]
    
    # Return the final output in the requested format
    result = {
        "state": random_state,
        "statuses": statuses
    }
    
    return result

# Test the function
result = get_random_state_status_and_timestamps()

# Output the result
import json
print(json.dumps(result, indent=2))


{
  "state": "Louisiana",
  "statuses": [
    {
      "status": "Shipped",
      "timestamp": "2022-07-02T03:31:02.064092Z"
    }
  ]
}


In [72]:
system_prompt_synthetic_nosql = """
<system_prompt>
YOU ARE A DATA GENERATOR EXPERT TASKED WITH CREATING SYNTHETIC JSON DATA OBJECTS. YOUR MISSION IS TO GENERATE REALISTIC, WELL-FORMATTED JSON OBJECTS THAT MIMIC SHIPMENT TRACKING INFORMATION.

###INSTRUCTIONS###

- The user will provide the `"SH_ID"` field and a list of status updates (e.g., `["Shipped", "In Transit", "Delivered"]`). You MUST use this exact `"SH_ID"` in the result object and NOT generate a new one. It should always be an INT value.
- GENERATE all other fields, including `"TR_ID"`, `"status_updates"` (with appropriate `"location"` and `"timestamp"` fields), `"estimated_delivery"`, and `"current_location"`.
- The `"status_updates"` field MUST contain **only the statuses** provided by the user in the input, keeping them in the same order (e.g., if the user provides `["Shipped", "In Transit"]`, you should only generate `"Shipped"` and `"In Transit"`).
  - DO NOT add extra statuses. If the final status is `"Delivered"`, ensure that `"Shipped"` and `"In Transit"` appear beforehand, but only if they were provided by the user.
  - Randomly ASSIGN locations (USA states) to each status update based on realistic travel distances between the locations and the time gap between timestamps.
  - Use the provided timestamps directly from the user's input. If not provided, generate realistic timestamps.
- The `"current_location"` field MUST reflect the location from the last status update provided by the user.
- The `"estimated_delivery"` field should reflect the date of the last status update if the `"Delivered"` status is present. If there is no `"Delivered"` status, set `"estimated_delivery"` to `null`.
- The `"TR_ID"` field MUST be a randomly generated string in the format of `"TRK"` followed by a random three-digit number (e.g., `"TRK001"`).

###Chain of Thoughts###

FOLLOW these steps in strict order to GENERATE the JSON object:

1. **RECEIVE SH_ID AND STATUSES**:
  - UTILIZE the `"SH_ID"` provided by the user directly in the JSON object without modification. It should always be an INT value.
  - ONLY USE the statuses provided by the user in the `"statuses"` field and ensure they appear in the correct order.

2. **GENERATE TR_ID**:
  - RANDOMLY GENERATE a `"TR_ID"` in the format `"TRK" + 3-digit number` (e.g., `"TRK001"`).

3. **HANDLE STATUS UPDATES**:
  3.1 USE the exact statuses provided by the user (e.g., `"Shipped"`, `"In Transit"`, `"Delivered"`) in the same order.
  3.2 ASSIGN random USA state cities to the `"location"` field for each status update.
  3.3 USE the exact timestamps provided by the user, or if missing, GENERATE realistic ISO timestamps that align with the sequence of status updates.

4. **GENERATE ESTIMATED DELIVERY AND CURRENT LOCATION**:
  - IF the `"Delivered"` status is present, SET the `"estimated_delivery"` field to the date of the `"Delivered"` status.
  - IF the `"Delivered"` status is NOT present, SET `"estimated_delivery"` to `null`.
  - SET the `"current_location"` field to the `"location"` of the most recent status update (the last in the list).

5. **FINALIZE THE JSON**:
  - ENSURE that all fields are populated according to the above logic.
  - STRUCTURE the JSON properly with correct formatting and syntax.

###What Not To Do###

- NEVER GENERATE A NEW `"SH_ID"`; ALWAYS USE THE ONE PROVIDED BY THE USER.
- NEVER ADD ADDITIONAL STATUSES THAT WERE NOT PROVIDED BY THE USER (e.g., do not add `"In Transit"` or `"Delivered"` if the user only provided `"Shipped"`).
- NEVER INCLUDE STATUS UPDATES THAT ARE OUT OF ORDER OR UNREALISTIC (e.g., `"Delivered"` before `"Shipped"`).
- NEVER USE LOCATIONS OUTSIDE THE USA.
- AVOID UNREALISTIC TIMESTAMPS (e.g., future dates or timestamps not in chronological order).
- DO NOT CREATE INCONSISTENCIES BETWEEN THE `"current_location"` AND THE LAST STATUS UPDATE.

###Few-Shot Example###

**Example 1:**

User input:
```json
{
  "SH_ID": 123,
  "state": "California",
  "statuses": [
    {
      "status": "Shipped",
      "timestamp": "2022-09-24T08:15:32.123456Z"
    },
    {
      "status": "In Transit",
      "timestamp": "2022-09-25T11:30:45.567890Z"
    },
    {
      "status": "Delivered",
      "timestamp": "2022-09-26T14:45:56.789012Z"
    }
  ]
}
```
Generated output:
```json
{
  "TR_ID": "TRK004",
  "SH_ID": 123,
  "status_updates": [
    {
      "status": "Shipped",
      "location": "Los Angeles, CA",
      "timestamp": "2022-09-24T08:15:32.123456Z"
    },
    {
      "status": "In Transit",
      "location": "Phoenix, AZ",
      "timestamp": "2022-09-25T11:30:45.567890Z"
    },
    {
      "status": "Delivered",
      "location": "Las Vegas, NV",
      "timestamp": "2022-09-26T14:45:56.789012Z"
    }
  ],
  "estimated_delivery": "2022-09-26",
  "current_location": "Las Vegas, NV"
}
```
**Example 2:**

User input:
```json
{
  "SH_ID": 456,
  "state": "Louisiana",
  "statuses": [
    {
      "status": "Shipped",
      "timestamp": "2022-07-02T03:31:02.064092Z"
    },
    {
      "status": "In Transit",
      "timestamp": "2022-07-03T10:10:45.678123Z"
    }
  ]
}
```

Generated output:
```json
{
  "TR_ID": "TRK025",
  "SH_ID": 456,
  "status_updates": [
    {
      "status": "Shipped",
      "location": "New Orleans, LA",
      "timestamp": "2022-07-02T03:31:02.064092Z"
    },
    {
      "status": "In Transit",
      "location": "Houston, TX",
      "timestamp": "2022-07-03T10:10:45.678123Z"
    }
  ],
  "estimated_delivery": null,
  "current_location": "Houston, TX"
}

```
</system_prompt>
"""

def process_data_gpt_json(user_input, system_prompt):
    chat_completion = client.chat.completions.create(
        messages=[{"role": "system", "content": system_prompt},
                  {
                      "role": "user",
                      # "content": user_prompt,
                      "content": user_input,
                  }
                  ],
        model="gpt-4o",
        response_format={"type": "json_object"},
        temperature=0.1,
        seed=123321
    )
    res = chat_completion.choices[0].message.content
    json_output_for_page = json.loads(res)
    return json_output_for_page

### Generate synthetic data

In [73]:
# Load the CSV file
file_path = 'data/Shipment_Details.csv'
df = pd.read_csv(file_path)

# Open the file in write mode
with open('output_data/synthetic_data_nosql.json', 'w') as f:
    # Loop through each row in the DataFrame
    for i in range(len(df)):
        # Extract the shipm ent ID
        input_data = {"SH_ID": df['SH_ID'].iloc[i]}
        
        # Get the random state and status with timestamps
        random_state_status = get_random_state_status_and_timestamps()
        print("Random data: ", random_state_status)
        # Merge the input_data (with SH_ID) and random state/status
        merged_data = {**input_data, **random_state_status}

        # Process the data using your custom function
        processed_data = process_data_gpt_json(str(merged_data), system_prompt_synthetic_nosql)
        
        # Output the processed data
        print(processed_data)

        print("Data processing complete.")
        f.write(json.dumps(processed_data) + "\n")  # Write the JSON object as a new line

print("Data processing complete, results written to synthetic_data_nosql.json")

Random data:  {'state': 'Indiana', 'statuses': [{'status': 'Shipped', 'timestamp': '2022-03-12T23:26:35.513755Z'}]}
{'TR_ID': 'TRK123', 'SH_ID': 690, 'status_updates': [{'status': 'Shipped', 'location': 'Indianapolis, IN', 'timestamp': '2022-03-12T23:26:35.513755Z'}], 'estimated_delivery': None, 'current_location': 'Indianapolis, IN'}
Data processing complete.
Random data:  {'state': 'New York', 'statuses': [{'status': 'Shipped', 'timestamp': '2021-04-10T07:33:21.151559Z'}]}
{'TR_ID': 'TRK123', 'SH_ID': 933, 'status_updates': [{'status': 'Shipped', 'location': 'New York, NY', 'timestamp': '2021-04-10T07:33:21.151559Z'}], 'estimated_delivery': None, 'current_location': 'New York, NY'}
Data processing complete.
Random data:  {'state': 'Alaska', 'statuses': [{'status': 'Shipped', 'timestamp': '2022-09-26T17:24:22.422531Z'}, {'status': 'In Transit', 'timestamp': '2022-09-27T12:45:20.181172Z'}, {'status': 'Delivered', 'timestamp': '2022-09-28T00:00:00Z'}]}
{'TR_ID': 'TRK123', 'SH_ID': 261, 

### Change the Track ID to random value

In [80]:
import json
import random

# Define the file paths
input_file_path = 'output_data/synthetic_data_nosql.ndjson'
output_file_path = 'output_data/modified_id_synthetic_data_nosql.ndjson'

# Function to randomly generate a TR_ID
def generate_random_tr_id():
    return "TRK" + str(random.randint(1000, 99999))

# Open the input NDJSON file and process each line
with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
    for line in infile:
        # Parse each line (which is a JSON object)
        if line.strip():  # Skip empty lines
            data = json.loads(line)

            # Replace the TR_ID field with a new random TR_ID
            if "TR_ID" in data and data["TR_ID"] == "TRK123":
                data["TR_ID"] = generate_random_tr_id()

            # Write the modified data back to the output file
            outfile.write(json.dumps(data) + "\n")

print("TR_ID fields have been successfully replaced with random values.")


TR_ID fields have been successfully replaced with random values.


In [81]:
input_file_path = 'output_data/modified_id_synthetic_data_nosql.ndjson'


def check_unique_tr_ids(file_path):
    tr_ids = set()  # Set to store unique TR_IDs
    duplicates = set()  # Set to store any duplicate TR_IDs

    with open(file_path, 'r') as infile:
        for line in infile:
            if line.strip():  # Skip empty lines
                data = json.loads(line)

                # Check if TR_ID exists in the object
                if "TR_ID" in data:
                    tr_id = data["TR_ID"]
                    # Check if the TR_ID is already in the set
                    if tr_id in tr_ids:
                        duplicates.add(tr_id)  # If it's a duplicate, add to duplicates set
                    else:
                        tr_ids.add(tr_id)  # Otherwise, add it to the unique set

    # Print results
    if len(duplicates) > 0:
        print("Duplicate TR_IDs found:")
        for tr_id in duplicates:
            print(tr_id)
    else:
        print("All TR_IDs are unique.")

# Call the function to check for unique TR_IDs
check_unique_tr_ids(input_file_path)

All TR_IDs are unique.
